## Step 1: importing the necessary packages and data

In [2]:
#install/import the ncecessary packages
import sys
assert sys.version_info >=(3, 5)

#import sklearn, tensorflow and keras
import sklearn
import tensorflow as tf
from tensorflow import keras

%load_ext tensorboard

#import other common packages
import numpy as np
import os


#set random seed
np.random.seed(42)

#specify matplotlib figures
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

#specify where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID =  "deep"
IMAGES_PATH =  os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print ("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
#import the CIFAR-10 dataset
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical

(X_train_full, y_train_full), (X_test, y_test) = cifar10.load_data()

In [3]:
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten

model = Sequential()
model.add(Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(Dense(100, activation="elu", kernel_initializer="he_normal"))
model.add(Dense(10, activation="softmax"))

#compile the model
optimizer = keras.optimizers.experimental.Nadam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 100)               307300    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 100)               1

In [5]:
#create callbacks

early_stopping_cb = keras.callbacks.EarlyStopping(patience=4)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 #increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks= [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [6]:
%tensorboard --logdir=./my_cifar_logs --port=6006

Reusing TensorBoard on port 6006 (pid 7824), started 0:20:32 ago. (Use '!kill 7824' to kill it.)

In [7]:
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=20, callbacks=callbacks)

Epoch 1/20
1407/1407 [==============================] - 74s 39ms/step - loss: 5.0268 - accuracy: 0.1549 - val_loss: 2.2351 - val_accuracy: 0.2028
Epoch 2/20
1407/1407 [==============================] - 50s 36ms/step - loss: 2.1382 - accuracy: 0.2168 - val_loss: 2.0647 - val_accuracy: 0.2312
Epoch 3/20
1407/1407 [==============================] - 54s 38ms/step - loss: 2.0277 - accuracy: 0.2527 - val_loss: 1.9915 - val_accuracy: 0.2490
Epoch 4/20
1407/1407 [==============================] - 63s 45ms/step - loss: 1.9529 - accuracy: 0.2811 - val_loss: 1.9401 - val_accuracy: 0.3038
Epoch 5/20
1407/1407 [==============================] - 58s 41ms/step - loss: 1.8790 - accuracy: 0.3149 - val_loss: 1.8543 - val_accuracy: 0.3072
Epoch 6/20
1407/1407 [==============================] - 63s 45ms/step - loss: 1.8214 - accuracy: 0.3382 - val_loss: 1.7750 - val_accuracy: 0.3508
Epoch 7/20
1407/1407 [==============================] - 58s 41ms/step - loss: 1.7713 - accuracy: 0.3583 - val_loss: 1.7807 -

In [8]:
#load model
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 2s 10ms/step - loss: 1.5933 - accuracy: 0.4284


[1.5933078527450562, 0.4284000098705292]

In [9]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [11]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=20,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/20
1407/1407 [==============================] - 140s 99ms/step - loss: 1.1719 - accuracy: 0.5879 - val_loss: 1.3735 - val_accuracy: 0.5222
Epoch 2/20
1407/1407 [==============================] - 112s 80ms/step - loss: 1.1563 - accuracy: 0.5914 - val_loss: 1.3322 - val_accuracy: 0.5354
Epoch 3/20
1407/1407 [==============================] - 116s 83ms/step - loss: 1.1408 - accuracy: 0.5986 - val_loss: 1.3393 - val_accuracy: 0.5326
Epoch 4/20
1407/1407 [==============================] - 71s 51ms/step - loss: 1.1096 - accuracy: 0.6076 - val_loss: 1.3263 - val_accuracy: 0.5454
Epoch 7/20
1407/1407 [==============================] - 92s 66ms/step - loss: 1.0294 - accuracy: 0.6397 - val_loss: 1.3928 - val_accuracy: 0.5318
Epoch 14/20
1407/1407 [==============================] - 120s 85ms/step - loss: 1.0187 - accuracy: 0.6421 - val_loss: 1.3546 - val_accuracy: 0.5370
Epoch 15/20
1407/1407 [==============================] - 102s 72ms/step - loss: 1.0044 - accuracy: 0.6443 - val_loss: 1

In [4]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [5]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>